<a href="https://colab.research.google.com/github/Nibi2003/Transfomer_keypoint/blob/main/Codigos/create_video_keypoints.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive

drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
!pip install moviepy


In [9]:
import cv2
from moviepy.editor import VideoFileClip

def get_video_info(video_path):
    """
    Obtém informações detalhadas do vídeo.

    Params:
    video_path: str
        Caminho para o arquivo de vídeo.

    Returns:
    dict
        Dicionário contendo várias informações sobre o vídeo.
    """
    # Use OpenCV to get basic information
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise ValueError("Erro ao abrir o vídeo.")

    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    duration = frame_count / fps

    cap.release()

    # Use moviepy to get additional information
    clip = VideoFileClip(video_path)
    video_codec = "N/A"  # Substitute a placeholder, as moviepy may not provide this directly
    audio_codec = "N/A"
    audio_bitrate = "N/A"
    audio_channels = "N/A"


    return {
        "fps": fps,
        "width": width,
        "height": height,
        "frame_count": frame_count,
        "duration": duration,

    }

# Caminho para o vídeo
video_path = '/content/drive/MyDrive/GMA/Base atual/Base atual dividida/teste/0/bebe07-1.mp4'

# Obter informações do vídeo
video_info = get_video_info(video_path)
print(f"FPS: {video_info['fps']}")
print(f"Largura: {video_info['width']}")
print(f"Altura: {video_info['height']}")
print(f"Número de Frames: {video_info['frame_count']}")
print(f"Duração: {video_info['duration']} segundos")



FPS: 30.0
Largura: 1080
Altura: 1920
Número de Frames: 900
Duração: 30.0 segundos


In [11]:
import cv2
import numpy as np
import os

class MediaPipeKeypointsVisualizer:
    def __init__(self):
        pass

    def draw_keypoints(self, keypoints, limbs=None, width=1080, height=1920):
        background = np.zeros((height, width, 3), dtype=np.uint8)

        # Desenha os keypoints no fundo preto
        for kp in keypoints:
            x, y = int(kp[0] * width), int(kp[1] * height)  # Escala para o tamanho da imagem
            cv2.circle(background, (x, y), 4, (0, 255, 0), -1)  # Desenha um círculo verde para cada ponto

        # Desenha as conexões entre os keypoints
        if limbs:
            for limb in limbs:
                start, end = limb
                if start < len(keypoints) and end < len(keypoints):
                    start_point = (int(keypoints[start][0] * width), int(keypoints[start][1] * height))
                    end_point = (int(keypoints[end][0] * width), int(keypoints[end][1] * height))
                    cv2.line(background, start_point, end_point, (0, 255, 255), 2)  # Desenha uma linha amarela para cada conexão

        return background

def create_video_from_keypoints(keypoints_file, out_path, fps=30, width=1080, height=1920, limbs=None):
    """
    Cria um vídeo mostrando os keypoints com fundo preto.

    Params:
    keypoints_file: str
        Caminho para o arquivo .npy contendo os keypoints.
    out_path: str
        Caminho onde o vídeo será salvo.
    fps: int, optional
        Frames por segundo do vídeo (default é 30).
    width: int, optional
        Largura do vídeo em pixels (default é 1080).
    height: int, optional
        Altura do vídeo em pixels (default é 1920).
    limbs: list of tuples, optional
        Lista de conexões entre os keypoints para desenhar os "ossos" (default é None).
    """
    # Carrega os keypoints
    keypoints = np.load(keypoints_file, allow_pickle=True)

    # Cria uma instância de MediaPipeKeypointsVisualizer
    visualizer = MediaPipeKeypointsVisualizer()

    # Inicializa o objeto VideoWriter
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec de vídeo
    video_writer = cv2.VideoWriter(out_path, fourcc, fps, (width, height))

    # Gera os frames e escreve no vídeo
    for frame in keypoints:
        frame_image = visualizer.draw_keypoints(frame, limbs=limbs, width=width, height=height)
        video_writer.write(frame_image)

    # Libera o objeto VideoWriter
    video_writer.release()

# Caminho para o arquivo .npy contendo os keypoints
keypoints_file = '/content/drive/MyDrive/GMA/Base atual/Kp_aumentados/0/bebe02-1_keypoints_aug_0.npy'

# Caminho para salvar o vídeo
out_path = '/content/drive/MyDrive/GMA/Base atual/Kp_aumentados/Videos/bebe02-1_keypoints_aug_0.mp4'

# Lista de conexões entre os keypoints (limbs)
limbs = [
    (0, 1), (1, 2), (2, 3), (3, 7), (0, 4), (4, 5), (5, 6), (6, 8),
    (9, 10), (11, 12), (11, 13), (13, 15), (15, 17), (15, 19), (15, 21),
    (17, 19), (12, 14), (14, 16), (16, 18), (16, 20), (16, 22), (18, 20),
    (11, 23), (12, 24), (23, 24), (23, 25), (25, 27), (27, 29), (27, 31),
    (29, 31), (24, 26), (26, 28), (28, 30), (28, 32), (30, 32)
]

# Criar o vídeo
create_video_from_keypoints(keypoints_file, out_path, fps=30, width=1080, height=1920, limbs=limbs)

print(f"Vídeo criado e salvo em {out_path}")


Vídeo criado e salvo em /content/drive/MyDrive/GMA/Base atual/Kp_aumentados/Videos/bebe02-1_keypoints_aug_0.mp4


In [12]:
#Excluir video criado
import os

# Caminho para o arquivo de vídeo que deseja excluir
video_path_to_delete = '/content/drive/MyDrive/GMA/Base atual/Kp_aumentados/Videos/bebe02-1_keypoints_aug_0.mp4'

# Verifica se o arquivo existe e o remove
if os.path.exists(video_path_to_delete):
    os.remove(video_path_to_delete)
    print(f"Vídeo {video_path_to_delete} foi excluído com sucesso.")
else:
    print(f"O arquivo {video_path_to_delete} não foi encontrado.")


Vídeo /content/drive/MyDrive/GMA/Base atual/Kp_aumentados/Videos/bebe02-1_keypoints_aug_0.mp4 foi excluído com sucesso.


In [14]:
import os
import glob
import numpy as np

data_dir = '/content/drive/MyDrive/GMA/Base atual/Kp_aumentados'

keypoints_list = []
labels_list = []

def process_keypoints(base_path, keypoints_list, labels_list):
    for class_folder in os.listdir(base_path):
        class_path = os.path.join(base_path, class_folder)

        for file_path in glob.glob(os.path.join(class_path, '*.npy')):
            keypoints = np.load(file_path, allow_pickle=True)
            keypoints_list.append(file_path)
            labels_list.append(int(class_folder))

process_keypoints(data_dir, keypoints_list, labels_list)


print("Número de arquivos de keypoints:", len(keypoints_list))
print("Número de rótulos:", len(labels_list))

index = 0
keypoints_file_path = keypoints_list[index]
keypoints = np.load(keypoints_file_path, allow_pickle=True)
label = labels_list[index]

print("Caminho do arquivo de keypoints:", keypoints_list)
print("Rótulo correspondente:", labels_list)


Número de arquivos de keypoints: 66
Número de rótulos: 66
Caminho do arquivo de keypoints: ['/content/drive/MyDrive/GMA/Base atual/Kp_aumentados/0/bebe05-2_keypoints_aug_0.npy', '/content/drive/MyDrive/GMA/Base atual/Kp_aumentados/0/bebe05-2_keypoints_aug_1.npy', '/content/drive/MyDrive/GMA/Base atual/Kp_aumentados/0/bebe05-2_keypoints_aug_2.npy', '/content/drive/MyDrive/GMA/Base atual/Kp_aumentados/0/bebe02-4_keypoints_aug_0.npy', '/content/drive/MyDrive/GMA/Base atual/Kp_aumentados/0/bebe02-4_keypoints_aug_1.npy', '/content/drive/MyDrive/GMA/Base atual/Kp_aumentados/0/bebe02-4_keypoints_aug_2.npy', '/content/drive/MyDrive/GMA/Base atual/Kp_aumentados/0/bebe02-2_keypoints_aug_0.npy', '/content/drive/MyDrive/GMA/Base atual/Kp_aumentados/0/bebe02-2_keypoints_aug_1.npy', '/content/drive/MyDrive/GMA/Base atual/Kp_aumentados/0/bebe02-2_keypoints_aug_2.npy', '/content/drive/MyDrive/GMA/Base atual/Kp_aumentados/0/bebe05-1_keypoints_aug_0.npy', '/content/drive/MyDrive/GMA/Base atual/Kp_aument

# Adiciona nas subpastas 0 e 1 dentro do diretorio de acordo com a label

> Precisa que o diretorio e os subdiretorios de 0 e 1 sejam previamente criados

In [16]:
def create_videos_for_keypoints(keypoints_list, labels_list, output_base_dir, fps=30, width=1080, height=1920, limbs=None):
    """
    Cria vídeos para uma lista de arquivos de keypoints, salvando-os em diretórios diferentes com base no rótulo.

    Params:
    keypoints_list: list of str
        Lista de caminhos para os arquivos .npy contendo os keypoints.
    labels_list: list of int
        Lista de rótulos correspondentes aos arquivos de keypoints.
    output_base_dir: str
        Diretório base onde os vídeos serão salvos.
    fps: int, optional
        Frames por segundo do vídeo (default é 30).
    width: int, optional
        Largura do vídeo em pixels (default é 1080).
    height: int, optional
        Altura do vídeo em pixels (default é 1920).
    limbs: list of tuples, optional
        Lista de conexões entre os keypoints para desenhar os "ossos" (default é None).
    """
    for kp_path, label in zip(keypoints_list, labels_list):
        # Define o diretório de saída com base no rótulo
        if label == 0:
            output_dir = os.path.join(output_base_dir, '0')
        elif label == 1:
            output_dir = os.path.join(output_base_dir, '1')
        else:
            print(f"Rótulo desconhecido {label} para {kp_path}. Ignorando.")
            continue

        # Certifique-se de que o diretório existe
        os.makedirs(output_dir, exist_ok=True)

        # Nome do arquivo de saída
        video_filename = os.path.join(output_dir, f'{os.path.splitext(os.path.basename(kp_path))[0]}.mp4')
        print(f"Criando vídeo para {kp_path} com rótulo {label}")

        # Cria o vídeo
        create_video_from_keypoints(kp_path, video_filename, fps=fps, width=width, height=height, limbs=limbs)

# Caminho base para os vídeos
output_base_dir = '/content/drive/MyDrive/GMA/Base atual/Kp_aumentados/Videos'

# Adiciona nas subpastas 0 e 1 dentro do diretorio de acordo com a label
# Criar vídeos
create_videos_for_keypoints(keypoints_list, labels_list, output_base_dir, fps=30, width=1080, height=1920, limbs=limbs)


Criando vídeo para /content/drive/MyDrive/GMA/Base atual/Kp_aumentados/0/bebe05-2_keypoints_aug_0.npy com rótulo 0
Criando vídeo para /content/drive/MyDrive/GMA/Base atual/Kp_aumentados/0/bebe05-2_keypoints_aug_1.npy com rótulo 0
Criando vídeo para /content/drive/MyDrive/GMA/Base atual/Kp_aumentados/0/bebe05-2_keypoints_aug_2.npy com rótulo 0
Criando vídeo para /content/drive/MyDrive/GMA/Base atual/Kp_aumentados/0/bebe02-4_keypoints_aug_0.npy com rótulo 0
Criando vídeo para /content/drive/MyDrive/GMA/Base atual/Kp_aumentados/0/bebe02-4_keypoints_aug_1.npy com rótulo 0
Criando vídeo para /content/drive/MyDrive/GMA/Base atual/Kp_aumentados/0/bebe02-4_keypoints_aug_2.npy com rótulo 0
Criando vídeo para /content/drive/MyDrive/GMA/Base atual/Kp_aumentados/0/bebe02-2_keypoints_aug_0.npy com rótulo 0
Criando vídeo para /content/drive/MyDrive/GMA/Base atual/Kp_aumentados/0/bebe02-2_keypoints_aug_1.npy com rótulo 0
Criando vídeo para /content/drive/MyDrive/GMA/Base atual/Kp_aumentados/0/bebe02-